In [ ]:
# Notwendige Installationen (nur einmal ausführen)
!pip install selenium  # Für die Steuerung des Browsers per Code
!pip install undetected-chromedriver  # Um Websiten zu scrapen, die Bot-Erkennung haben
!pip install webdriver-manager  # Automatisches Herunterladen des passenden ChromeDrivers
!pip install openpyxl  # Ermöglicht das Lesen/Schreiben von Excel-Dateien

In [ ]:
#  Erstelle einen Chrome-Browser im Hintergund (headless)
import undetected_chromedriver as uc
driver = uc.Chrome(headless=True, use_subprocess=False)

In [ ]:
# weitere Importe
import time
import pandas as pd
from bs4 import BeautifulSoup  # für HTML-Parsing
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# Dateien und Konfiguration
eingabe_datei = "tandfonline_journals.xlsx"  # Excel-Datei mit URLs
ziel_datei = "tandfonline_journals_final.xlsx"  # Zieldatei mit den extrahierten Metriken
delay = 5  # Wartezeit in Sekunden zwischen den Anfragen

# Lade die Eingabedatei mit Pandas
df = pd.read_excel(eingabe_datei)

In [ ]:
# Funktion zum Erstellen eines Selenium Webdrivers mit Chrome
def create_driver():
    options = Options()
    options.add_argument("--no-sandbox")  # verhindert Sandbox-Probleme (v.a. bei Servern)
    options.add_argument("--window-size=1920,1080")  # setzt Fenstergröße
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

In [ ]:
# Funktion zum Extrahieren der Metriken von einer Journal-Seite
def extract_metrics_with_selenium(url):
    driver = create_driver()  # Erzeuge einen neuen Browser für jede URL
    try:
        driver.get(url)   # Lade die Website
        time.sleep(5)  # Zeit zum vollständigen Laden der Seite

        # HTML der Seite analysieren
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Suche nach dem Container, der die Metriken enthält
        container = soup.find("div", class_="journalMetrics")
        if not container:
            return {"Fehler": "journalMetrics-Container nicht gefunden"}

        metrics = {}

        # Usage-Metrik extrahieren
        usage_div = container.find("div", class_="usage")
        if usage_div:
            li = usage_div.find("li")
            if li:
                strong_tag = li.find("strong")
                if strong_tag:
                    value = strong_tag.get_text(strip=True)  # Der Wert (z.B. "100.000 Downloads") 
                    key = li.get_text(strip=True).replace(value, "").strip()  # Der Schlüssel (z.B. "Usage")
                    metrics[key] = value
                else:
                    metrics["Usage"] = li.get_text(strip=True)
        # Citation-Metriken extrahieren
        citation_div = container.find("div", class_="citation-metrics")
        if citation_div:
            for li in citation_div.find_all("li"):
                strong_tag = li.find("strong")
                if strong_tag:
                    value = strong_tag.get_text(strip=True)
                    key = li.get_text(strip=True).replace(value, "").strip()
                    metrics[key] = value
                else:
                    metrics["Citation"] = li.get_text(strip=True)

        # Speed/Acceptance-Metriken extrahieren
        speed_div = container.find("div", class_="speed")
        if speed_div:
            for li in speed_div.find_all("li"):
                strong_tag = li.find("strong")
                if strong_tag:
                    value = strong_tag.get_text(strip=True)
                    key = li.get_text(strip=True).replace(value, "").strip()
                    metrics[key] = value
                else:
                    metrics["Speed/Acceptance"] = li.get_text(strip=True)

        return metrics  # Rückgabe der gesammelten Metriken als Dictionary

    except Exception as e:
        return {"Fehler": str(e)}  # Rückgabe einer Fehlermeldung bei Problemen

    finally:
        driver.quit()  # Browser schließen, egal ob Erfolg oder Fehler

In [ ]:
# Liste zum Speichern aller extrahierten Metriken
alle_metrics = []

# Iteriere durch jede Zeile der Excel-Datei
for i, row in df.iterrows():
    url = row.get("URL")  # Hole die URL aus der Spalte "URL"
    print(f"[{i+1}/{len(df)}] Scraping: {url}")
    metrics = extract_metrics_with_selenium(url)  # Extrahiere Metriken für diese URL
    alle_metrics.append(metrics)  # Füge das Ergebnis zur Liste hinzu
    print(f"Fertig mit URL {i+1}. Warte {delay} Sekunden...\n")
    time.sleep(delay)  # Verzögerung zwischen den Anfragen, um den Server nicht zu überlasten

In [ ]:
# Erstelle ein neues DataFrame aus den extrahierten Metriken
metrics_df = pd.DataFrame(alle_metrics)
# Kombiniere ursprüngliche Daten mit den neuen Metriken
df_out = pd.concat([df, metrics_df], axis=1)
# Speichere das Ergebnis in eine neue Excel-Datei
df_out.to_excel(ziel_datei, index=False)

print(f"Fertig! Datei gespeichert unter: {ziel_datei}")